In [204]:
"""
SMS Template Magic

first_name
last_name
key_date
key_time
response_phone_number
today (+/-)
custom_fields (json key)
"""

sample_templates = """
"Don't forget about your healthcare appointment today at [time]. Please arrive 10 minutes early and bring along your ID and insurance card. If you need to reschedule, please call [office phone number]. We look forward to seeing you soon!"
"""

In [1]:
import csv
import os
import sqlite3

import pandas as pd

table_csv_path = './data/table_csvs'
schema_files = []
data_files = []

for folder, subfolders, files in os.walk(table_csv_path):
    for filename in files:
        if filename.endswith('_data.csv'):
            data_files.append(filename)
        elif filename.endswith('.csv'):
            schema_files.append(filename)
            
for schema_filename in schema_files:
    table_name = schema_filename.rsplit('.csv', 1)[0]
    data_filename = f'{table_name}_data.csv'
    df = pd.read_csv(f'{table_csv_path}/{schema_filename}')
    df = df.unstack().unstack()
    df.columns = ['col_type']
    
    create_statement_rows = []
    insert_statement_rows = []
    
    drop_statement = f'DROP TABLE IF EXISTS {table_name};'
    for i in df.index:
        create_statement_rows.append(f"    {i} {df.loc[i,'col_type']}")
    create_statement_rows = ',\n'.join(create_statement_rows)
    create_statement = f'CREATE TABLE {table_name} (\n{create_statement_rows}\n);'

    if data_filename in data_files:
        with open(f'{table_csv_path}/{data_filename}', newline='') as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                row = "', '".join(row)
                row = f"('{row}')"
                insert_statement_rows.append(row)
        insert_statement_rows = insert_statement_rows[1:]
        insert_statement_rows = ',\n'.join(insert_statement_rows)
        insert_statement = f'INSERT INTO {table_name} VALUES\n{insert_statement_rows}\n;'
    else:
        insert_statement = ''
    
    with sqlite3.connect('./data/test_db.db') as conn:
        cur = conn.cursor()
        print(drop_statement)
        cur.execute(drop_statement)
        print(create_statement)
        cur.execute(create_statement)
        print(insert_statement)
        cur.execute(insert_statement)
        conn.commit()



DROP TABLE IF EXISTS templates;
CREATE TABLE templates (
    id INTEGER,
    account_id INTEGER,
    name TEXT,
    template_type TEXT,
    workflow_category TEXT,
    active TEXT
);
INSERT INTO templates VALUES
('1', '1', 'Same Day Appt Reminder', 'nodes.outreach.SMSOutreach', 'Appointment Reminders', 'TRUE'),
('2', '1', 'Prescription Reminder', 'nodes.outreach.SMSOutreach', 'Prescription Reminders', 'TRUE'),
('3', '2', 'Same Day Appt Reminder', 'nodes.outreach.SMSOutreach', 'Appointment Reminders', 'TRUE'),
('4', '2', 'Prescription Reminder', 'nodes.outreach.SMSOutreach', 'Prescription Reminders', 'TRUE')
;
DROP TABLE IF EXISTS workflows;
CREATE TABLE workflows (
    id INTEGER,
    account_id INTEGER,
    name TEXT,
    workflow_category TEXT,
    active TEXT
);

DROP TABLE IF EXISTS members;
CREATE TABLE members (
    id INTEGER,
    account_id INTEGER,
    provider_id TEXT,
    first_name TEXT,
    last_name TEXT,
    phone_number INTEGER,
    email TEXT,
    key_date TEXT,
    ke

In [2]:
# !pip3 freeze